#For now, Faiss-RAG approach implemented partially, only for robots no convertion to dialog from vectors

Libraries import, downgrade numpy to make libs work together


In [ ]:
# !pip install faiss-cpu
# !pip install gensim
# !pip install pybind11
# !pip install numpy<2
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_KEY")

from huggingface_hub import login

login(token=secret_value_0)

In [ ]:
import faiss # our agent to find connection between vectors
from gensim.models import Word2Vec # conversion words into vectors
import numpy as np # math

import nltk # to convert text into some more presentable way
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
mistral = "mistralai/Mistral-7B-Instruct-v0.3"
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    )
mistral_tokenizer = AutoTokenizer.from_pretrained(mistral)


In [ ]:
def file_convert(file_name):
  with open(file_name, "r", encoding="utf-8") as file: ## reading a file and storing it into one variable
    f = file.read()

  sentenses = sent_tokenize(f) # our english teacher
  tokenized_sentenses = [word_tokenize(sent) for sent in sentenses] # structure a text

  model = Word2Vec(tokenized_sentenses, vector_size=100, window=5, min_count=2, sg=1) # convertation to vectors
  document_vectors = [] # TODO

  for sentence in tokenized_sentenses:
      sentence_vector = np.mean([model.wv[word] for word in sentence if word in model.wv], axis=0)
      if np.isnan(sentence_vector).any():
          sentence_vector = np.zeros(model.vector_size)
      document_vectors.append(sentence_vector)
  return document_vectors, model, tokenized_sentenses


In [ ]:
doc_vec, model, dataset = file_convert("/kaggle/input/church-s/church_text")
#print(doc_vec[:1])

# from sklearn.preprocessing import normalize -> 4 decimal difference (less than 1%)
# norm_vec = normalize(doc_vec, axis=1)
# print(norm_vec[:1])

## create a function to chunk words into 50-100 words, instead of just one, we can't change it directly in
## the function file_convert because we need to build document vectors

def chunk_tokenized_sentences(tokenized_sentences, chunk_size=100, stride=20):
    chunks = []
    current_chunk = []
    current_length = 0

    flat_words = [word for sentence in tokenized_sentences for word in sentence]

    for i in range(0, len(flat_words), chunk_size - stride):
        chunk = flat_words[i:i + chunk_size]
        chunks.append(" ".join(chunk))

    return chunks

chunk = chunk_tokenized_sentences(dataset, chunk_size=50)

In [ ]:
def faiss_search(document, model): # now we initialize a search on a given field
  document_vectors = np.array(document).astype("float32")

  index = faiss.IndexFlatIP(model.vector_size)
  index.add(document_vectors)  # add vectors to FAISS
  return index, model

In [ ]:
faiss_index, faiss_model = faiss_search(doc_vec,model)

In [ ]:
question = "Larnaka"

sntc_question = sent_tokenize(question)
tokenized_question = [word_tokenize(sent) for sent in sntc_question]

# vectorization with numpy
words = [model.wv[word] for word in tokenized_question[0] if word in model.wv]

if not words:
    question_vector = np.zeros(model.vector_size)
else:
    question_vector = np.mean(words, axis=0)

query = question_vector.reshape(1, -1).astype("float32")
query[:1]

In [ ]:
# faiss search
distances, indices = faiss_index.search(query, k=3)

threshold = 0.3 # ограничение по "релевантности" запроса, насколько соотвествует ожиданиям, сортировка мусора

valid_indices = indices[distances > threshold]
valid_distances = distances[distances > threshold]

# context = [dataset[i] for i in valid_indices]

if len(valid_indices) == 0:
    print("trash values")
else:
    # print(f"relevant info: {context}")
    print(f"in range: {valid_distances}")

